# Get Started With Reasoning Models

This is your first introduction to working with reasoning models, code-first. In this notebook, we'll primarily use the o4-mini model. However, we will also explore the `o1` model briefly, to understand how the API works for both. Use [this table](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python-secure%2Cpy#api--feature-support) for the latest information on supported features. And visit the [Reasoning Models](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/reasoning?tabs=python%2Cpy) documentation page for API details and code snippets. Here's a quick summary for convenience:

<br/>

| Characteristic | o1 | o4-mini |
|:--- |:---|:---|
| Developer Messages    | ✅ | ✅ |
| Structured Outputs    | ✅ | ✅ |
| Context Window Input  | 200K | 100K |
| Context Window Output | 200K | 100K |
| Reasoning Effort      | ✅ | ✅ |
| Vision Support        | ✅ | ✅ |
| Chat Completions API  | ✅ | ✅ |
| Responses API         | ✅ |    |
| Functions / Tools     | ✅ | ✅ |
| max_completion_tokens | ✅ |    |
| System messages       | ✅ | ✅ |
| Reasoning summary     | ✅ | |
| Streaming             | ✅ | |
| Model Card | [o4-mini](https://ai.azure.com/explore/models/o4-mini/version/2025-04-16/registry/azure-openai) | [o1](https://ai.azure.com/explore/models/o1/version/2024-12-17/registry/azure-openai)  |
| api_version | 2025-04-01-preview | 2025-03-01-preview |

---

## 1. Quickstart

In this section, we'll do a quick check to make sure we have the right dependencies installed. We'll also test both o1 and o4-mini models but we'll primarily use the `o4-mini` unless explicitly stated otherwise.

### 1.1 Install Python Dependencies

In [ ]:
# Install and Upgrade Pip
!pip install --upgrade pip --quiet

# Install Required Packages
!pip install -q openai azure-identity python-dotenv --quiet

# You may need to updated your OpenAI Python library
!pip install openai --upgrade --quiet


### 1.2 Check Env Variables

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

if not os.getenv("AZURE_OPENAI_ENDPOINT"):
    print("Missing env: AZURE_OPENAI_ENDPOINT")
elif not os.getenv("AZURE_OPENAI_KEY"):
    print("Missing env: AZURE_OPENAI_KEY")
else: 
    print("Azure OpenAI endpoint and key are set.")

### 1.3 Define Utility Functions

In [ ]:
# Print stats for the response
def pretty_print(response, response_time):
    """
    Prints the response details in a formatted manner.
    Args:
        response (openai.types.chat.chat_completion.ChatCompletion): The response object containing the generated completion.
        response_time (float): The time taken to get the response.
    """
    print(".........................")
    print(f"Response time: {response_time:.2f} seconds")
    print(f"Total Tokens: {response.usage.total_tokens}")
    print(f"Prompt Tokens: {response.usage.prompt_tokens}")
    print(f"Completion Tokens: {response.usage.completion_tokens}")
    print(f"Reasoning Tokens: {response.usage.completion_tokens_details.reasoning_tokens}")
    print(f"Output Tokens: {response.usage.total_tokens - response.usage.completion_tokens_details.reasoning_tokens}")
    print(f"\nResponse:\n {response.choices[0].message.content}")
    print(f".........................")


In [15]:

from openai import AzureOpenAI
import time

# Chat with o1
def o1_chat(prompt="hi", reasoning_level="medium", developer_message="You are a helpful assistant"):
    """
    Sends a chat completion request to the Azure OpenAI API o1 model.
    Args:
        prompt (str): The input prompt to generate a response for.
        reasoning_level (str): The reasoning effort level ('low', 'medium', 'high').
        developer_message (str): The developer message to set the context for the assistant.
    Returns:
        response (openai.types.chat.chat_completion.ChatCompletion): The response object containing the generated completion.
    """
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_key=os.getenv("AZURE_OPENAI_KEY"),  
        api_version="2025-04-01-preview"
    )
    try:
        request_time = time.time()
        response = client.chat.completions.create(
            model="o1",
            messages=[
                {"role": "developer", "content": developer_message},
                {"role": "user", "content": prompt},
            ],
            max_completion_tokens=5000,
            reasoning_effort=reasoning_level
        )
        response_time = time.time() - request_time
        pretty_print(response, response_time)
        return response

    except Exception as e:
        print(f"[ERROR] {e}")
        return None


In [16]:
# Chat with o4_mini
def o4mini_chat(prompt="hi", reasoning_level="medium", developer_message="You are a helpful assistant", response_format=None):
    """
    Sends a chat completion request to the Azure OpenAI API o4-mini model.
    Args:
        prompt (str): The input prompt to generate a response for.
        reasoning_level (str): The reasoning effort level ('low', 'medium', 'high').
        developer_message (str): The developer message to set the context for the assistant.
        response_format (BaseModel, optional): The expected structured output format for the response.
    Returns:
        response (openai.types.chat.chat_completion.ChatCompletion): The response object containing the generated completion.
    """
    client = AzureOpenAI(
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_key=os.getenv("AZURE_OPENAI_KEY"),  
        api_version="2025-04-01-preview"
    )
    try:
        request_time = time.time()
        response = client.chat.completions.create(
            model="o4-mini",
            messages=[
                {"role": "developer", "content": developer_message},
                {"role": "user", "content": prompt},
            ],
            max_completion_tokens=5000,
            reasoning_effort=reasoning_level,
            response_format=response_format
        )
        response_time = time.time() - request_time
        pretty_print(response, response_time)
        return response

    except Exception as e:
        print(f"[ERROR] {e}")
        return None

## 2.Test Models

In [17]:
# Test o1
response = o1_chat(
    prompt="How many p's in hippopotamus?",
    reasoning_level="medium",
    developer_message="You are a helpful assistant."
)

.........................
Response time: 3.09 seconds
Total Tokens: 307
Prompt Tokens: 25
Completion Tokens: 282
Reasoning Tokens: 256
Output Tokens: 51

Response:
 There are 3 p’s in "hippopotamus."
.........................


In [18]:
# Test o4mini
response = o4mini_chat(
    prompt="How many p's in hippopotamus?",
    reasoning_level="low",
    developer_message="You are a helpful assistant."
)

.........................
Response time: 3.58 seconds
Total Tokens: 190
Prompt Tokens: 25
Completion Tokens: 165
Reasoning Tokens: 128
Output Tokens: 62

Response:
 There are 3 letter “p”s in the word “hippopotamus.”
.........................


---

## 2. Let's Prompt!

In [ ]:
# -------------------
# MATH EXAMPLE
# -------------------
response = o4mini_chat(
    prompt="A train travels at 60 mph. How long does it take to travel 90 miles?",
    reasoning_level="low",
    developer_message="You are a math tutor. Explain the solution"
)
print(response.choices[0].message.content)

.........................
Response time: 3.86 seconds
Total Tokens: 217
Prompt Tokens: 38
Completion Tokens: 179
Reasoning Tokens: 64
Output Tokens: 153

Response:
 To find the time, use the relationship  
   time = distance ÷ speed  

Here, distance = 90 miles and speed = 60 miles per hour, so  
   time = 90 miles ÷ 60 (miles/hour)  
        = 1.5 hours  

Since 0.5 hour = 30 minutes, 1.5 hours = 1 hour 30 minutes.  
Answer: It takes 1 hour and 30 minutes.
.........................
To find the time, use the relationship  
   time = distance ÷ speed  

Here, distance = 90 miles and speed = 60 miles per hour, so  
   time = 90 miles ÷ 60 (miles/hour)  
        = 1.5 hours  

Since 0.5 hour = 30 minutes, 1.5 hours = 1 hour 30 minutes.  
Answer: It takes 1 hour and 30 minutes.


In [21]:
# -------------------
#  MATH REASONING
# -------------------
prompt = "Jane has twice as many apples as Tom. Together they have 18 apples. How many does each person have?"
response = o4mini_chat(prompt)
print(response.choices[0].message.content)

.........................
Response time: 3.95 seconds
Total Tokens: 247
Prompt Tokens: 38
Completion Tokens: 209
Reasoning Tokens: 128
Output Tokens: 119

Response:
 Let t = the number of apples Tom has.  
Then Jane has 2t apples, and together:  
t + 2t = 18  
3t = 18  
t = 6  

So Tom has 6 apples, and Jane has 2·6 = 12 apples.
.........................
Let t = the number of apples Tom has.  
Then Jane has 2t apples, and together:  
t + 2t = 18  
3t = 18  
t = 6  

So Tom has 6 apples, and Jane has 2·6 = 12 apples.


In [22]:
# -------------------
#  SCIENCE REASONING
# -------------------
prompt = "Why does a metal spoon feel colder than a wooden spoon when left in the same room?"
response = o4mini_chat(prompt)
print(response.choices[0].message.content)

.........................
Response time: 13.93 seconds
Total Tokens: 924
Prompt Tokens: 33
Completion Tokens: 891
Reasoning Tokens: 512
Output Tokens: 412

Response:
 Even though both spoons are at “room temperature,” your skin feels them very differently simply because of how fast they conduct heat.  Here’s what’s going on:

1. Thermal conductivity  
   - Metals (e.g. steel, silver) have very high thermal conductivity.  Heat moves through them quickly.  
   - Wood is a poor conductor (an insulator), so heat moves very slowly.

2. Heat flow and what your nerves detect  
   - When you touch an object warmer or cooler than your skin, heat will flow across that contact.  
   - Thermoreceptors in your skin respond primarily to the rate of heat transfer, not just the temperature difference.  
   - With the metal spoon, heat rushes out of your warm hand into the colder spoon faster, so your receptors register a strong “cold” sensation.  
   - With the wooden spoon, heat still flows from your

In [23]:
# -------------------
#  MULTI-STEP PLANNING
# -------------------
prompt = "Design a basic weekly study schedule to learn Python in 6 weeks, assuming 1 hour per weekday."
response = o4mini_chat(prompt)
print(response.choices[0].message.content)

.........................
Response time: 76.42 seconds
Total Tokens: 1760
Prompt Tokens: 36
Completion Tokens: 1724
Reasoning Tokens: 1024
Output Tokens: 736

Response:
 Here’s a simple 6‑week plan (Mon–Fri, 1 h/day = 30 h total) to get you from zero to a small Python project.

Week 1 – Setup & Core Syntax  
 Mon 1: Install Python & IDE (VS Code, PyCharm or Thonny), run “Hello, world!”  
 Tue 2: Variables, basic data types (int, float, str, bool)  
 Wed 3: Numeric operations, type conversion, simple expressions  
 Thu 4: String basics: indexing, slicing, common methods (.upper, .split, f‑strings)  
 Fri 5: Hands‑on exercises (e.g. build a simple text‑based calculator)

Week 2 – Control Flow  
 Mon 6: Boolean logic, comparison operators  
 Tue 7: if, elif, else statements  
 Wed 8: for loops (iterating lists, ranges)  
 Thu 9: while loops and loop control (break, continue)  
 Fri 10: Practice challenge (e.g. FizzBuzz, guessing game)

Week 3 – Built‑in Data Structures  
 Mon 11: Lists: c

In [24]:
# --------------------------------
# CONSTRAINT-BASED SCHEDULING
# --------------------------------
prompt = "Three employees—Alex, Sam, and Riley—must each work one 4-hour shift today. Only Alex and Riley can work before noon, and Sam can’t work past 4 PM. Create a valid schedule."
response = o4mini_chat(prompt)
print(response.choices[0].message.content)

.........................
Response time: 17.66 seconds
Total Tokens: 871
Prompt Tokens: 59
Completion Tokens: 812
Reasoning Tokens: 704
Output Tokens: 167

Response:
 Here’s one way to satisfy all the constraints (each works a single 4‑hour block, only Alex/Riley ever work before 12, and Sam finishes by 4 PM):

• 8:00 AM – 12:00 PM: Alex  
• 12:00 PM – 4:00 PM: Sam  
• 4:00 PM – 8:00 PM: Riley
.........................
Here’s one way to satisfy all the constraints (each works a single 4‑hour block, only Alex/Riley ever work before 12, and Sam finishes by 4 PM):

• 8:00 AM – 12:00 PM: Alex  
• 12:00 PM – 4:00 PM: Sam  
• 4:00 PM – 8:00 PM: Riley


In [ ]:
# -------------------
# TRY YOUR OWN
# -------------------

# Write your own prompt here
prompt = ""  

# Change these if you want to experiment
reasoning_level = "medium"
developer_message = "You are a helpful assistant."
response = o4mini_chat(prompt, reasoning_level, developer_message)
print(response.choices[0].message.content)